In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from src.utils import save_figure

# Get datasets

In [2]:
me_snrs = [10, 80]
white_snrs = [-10, 0, 10, 20, 50, 80]
colors = {str(white_snrs[i]):plt.cm.tab10.colors[i] for i in range(len(white_snrs))}

def get_dataset(dataset_path):
    X = np.load(os.path.join(dataset_path, "X.npy"))
    y_reg = np.load(os.path.join(dataset_path, "y_reg.npy"))
    return X, y_reg

datasets = {}

for dataset in os.listdir("../simulated_data/"):
    if dataset.startswith("DS"):
        X, y_reg = get_dataset(f"../simulated_data/{dataset}")
        datasets[dataset] = {'X': X, 'y_reg':y_reg}

datasets

{'DS_50_80_10': {'X': array([[ 1.20272845,  0.94814687, -0.01941026, ...,  0.91122791,
           1.13149112, -1.42632913],
         [-1.05345942, -0.826725  ,  1.02326702, ..., -1.01326169,
           0.57458985, -0.23213286],
         [-1.33017969, -0.10472354, -1.42020851, ..., -0.82906527,
          -0.98683143,  0.61397583],
         ...,
         [ 0.20603958,  4.00717278,  5.68563457, ...,  1.83641198,
          -0.07614077, -0.4907831 ],
         [ 1.67096401, -0.45554884,  0.49215176, ...,  0.41861727,
          -0.47619474, -0.23007506],
         [-0.60257223, -0.94035939, -2.026471  , ...,  0.11733745,
           2.27113914, -1.45195815]]),
  'y_reg': array([ 0.,  0.,  0., ..., 66., 52., 92.])},
 'DS_20_80_10': {'X': array([[ 3.87412674e+01,  3.06581708e+01, -2.37286493e-02, ...,
           2.82294295e+01,  3.51543497e+01, -4.57574594e+01],
         [-3.26056088e+01, -2.54662266e+01,  3.29509311e+01, ...,
          -3.26286733e+01,  1.75452740e+01, -7.99350858e+00],
        

# Distribution

In [3]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

# Function to calculate summary statistics for each dataset
def summarize_statistics(data):
    stats_summary = {}
    
    # Flatten the data to compute statistics across all points
    flattened_data = data.flatten()
    
    # Mean
    stats_summary['Mean'] = np.mean(flattened_data)
    
    # Standard Deviation
    stats_summary['Standard Deviation'] = np.std(flattened_data)
    
    # Minimum and Maximum (Range)
    stats_summary['Min'] = np.min(flattened_data)
    stats_summary['Max'] = np.max(flattened_data)
    
    # Median
    stats_summary['Median'] = np.median(flattened_data)
    
    # IQR (Interquartile Range)
    stats_summary['IQR'] = stats.iqr(flattened_data)
    
    # Kurtosis
    stats_summary['Kurtosis'] = stats.kurtosis(flattened_data, fisher=True)
    
    # Skewness
    stats_summary['Skewness'] = stats.skew(flattened_data)
    
    return stats_summary


# List to store summary statistics for all datasets
summary_stats = []

# Calculate summary statistics for each dataset
for dataset_name, data in datasets.items():
    stats_summary = summarize_statistics(data['X'])  # Compute stats for X
    stats_summary['Dataset'] = dataset_name  # Add dataset name for identification
    summary_stats.append(stats_summary)

# Calculate overall statistics by combining all datasets
all_data = np.concatenate([data['X'].flatten() for data in datasets.values()])
overall_stats = summarize_statistics(all_data)
overall_stats['Dataset'] = 'Overall Average'
summary_stats.append(overall_stats)

# Convert summary stats to a DataFrame for easier viewing
summary_df = pd.DataFrame(summary_stats).set_index('Dataset')
print(summary_df)

# Save the DataFrame as a CSV file for further analysis if needed
summary_df.to_csv("summary_statistics.csv")


                     Mean  Standard Deviation           Min           Max  \
Dataset                                                                     
DS_50_80_10     -0.178780            7.534345   -192.509965    101.035505   
DS_20_80_10     -0.170651           51.029826   -355.259220    339.037268   
DS_0_10_10       0.105770          529.275518  -3447.420470   3500.223988   
DS_-10_10_10     0.287296         1604.593704 -10978.428015  10486.249792   
DS_50_10_10      0.022084          158.869960   -878.741700    629.892750   
DS_80_10_10      0.021827          158.862692   -879.108852    632.962376   
DS_0_80_10      -0.095094          505.003124  -3502.842024   3386.013627   
DS_20_10_10      0.030214          166.671285  -1086.727696    870.503962   
DS_80_80_10     -0.179038            7.363159   -192.454945     99.592105   
DS_10_80_10     -0.152498          159.849381  -1107.674582   1070.703633   
DS_-10_80_10     0.086432         1596.805175 -11077.026521  10707.666697   

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Create subplots: one for each ME SNR
fig, axes = plt.subplots(1, 2)  # Two subplots side by side

for i, me_snr in enumerate(me_snrs):
    sns.set(style="whitegrid")
    
    for j, white_snr in enumerate(white_snrs):
        dataset_key = f"DS_{white_snr}_{me_snr}_10"
        dataset_data = datasets[dataset_key]["X"].flatten()  # Flatten data for KDE plot
        
        # Plot on the appropriate subplot
        sns.kdeplot(
            dataset_data, 
            lw=2, 
            label=f"White SNR: {white_snr}", 
            color=colors[str(white_snr)], 
            ax=axes[i]
        )
    
    # Set plot details for the current subplot
    axes[i].set_title(f'ME SNR: {me_snr}')
    axes[i].set_xlabel(r'Signal Amplitude')
    axes[i].set_ylabel(r'Density')
    axes[i].set_xlim(-4000, 4000)
    axes[i].set_ylim(0, 0.004)
    if i == 1: 
        axes[i].legend(title='White noise SNR', loc='center right', bbox_to_anchor=(1.6, 0.5))

# Adjust layout and save the figure
        
fig.suptitle('Distributions Across White Noise and ME SNRs', 
    y=0.95)
fig.subplots_adjust(left=0.05, right=0.95, top=0.85, bottom=0.1, wspace=0.7)
save_figure('distributions_me_snr', width=6, height=4)


# PSD

In [6]:
from scipy.signal import welch

In [11]:
num_freq_to_show = 200
fs = 30000

# get tab:10 from plt

fig, axs = plt.subplots(1, 2, figsize=(15, 5))
fig.set_size_inches(7.5,6)
lines = []
for i, me_snr in enumerate(me_snrs):

    for j, white_snr in enumerate(white_snrs):
        average_Pxx = np.zeros(512//2 + 1)
        for sample_id in range(3000):
            X = datasets[f"DS_{white_snr}_{me_snr}_10"]["X"]
            f, Pxx = welch(X[sample_id], fs, nperseg=512)
            Pxx = Pxx/Pxx.max()
            average_Pxx += Pxx
        average_Pxx /= 10
        line = axs[i].plot(f[:num_freq_to_show], average_Pxx[:num_freq_to_show] + 200*j, label=white_snr, c=colors[str(white_snr)])
        lines.append(line[0])
        axs[i].set_title(f"ME_SNR = {me_snr}")
        axs[i].set_ylim([0, 1300])
        axs[i].set_yticks([])

fig.legend(lines[-6:], white_snrs,
    title='White SNR', loc='center right', bbox_to_anchor=(1.1, 0.5))

save_figure("PSD_real_data", width=6)
plt.show()